In [2]:
# main imports 

import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from tqdm import tqdm
import pickle

from tqdm import tqdm
import seaborn as sns

import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential      #This allows appending layers to existing models
from tensorflow.keras.layers import Dense           #This allows defining the characteristics of a particular layer
from tensorflow.keras import optimizers             #This allows using whichever optimiser we want (sgd,adam,RMSprop)
from tensorflow.keras import regularizers           #This allows using whichever regularizer we want (l1,l2,l1_l2)
from tensorflow.keras.utils import to_categorical   #This allows using categorical cross entropy as the cost function
#from tensorflow.keras import Conv2D
#from tensorflow.keras import MaxPooling2D
#from tensorflow.keras import Flatten

from sklearn.model_selection import train_test_split
from numpy import asarray

In [3]:
#main functions

def get_path(Id,df):
    """Input: row number of the dataframe
       Returns the filepath for the corresponding image"""
    
    #get path from dataframe
    path2 = df.loc[Id]["filepaths"]
    
    #return the path
    return path2

def get_image(Id,df,x=224,y=224):
    """Returns image of a given row of the birds dataframe"""
    #get initial path to all data
    path1 = "kaggle/input/inputbirds/"
    
    #call fet_path function
    path2 = get_path(Id,df)
    
    #read image of the path1+path2 and return it
    return Image.open(path1+path2).resize((x,y))

def get_input_NN(Id,df,gray = False):
    """Return a 3D matrix of image ready to be read in the CNN"""
    
    #call fet_image function to read image
    im = get_image(Id,df)
    
    #transform iamge to RGB format
    rgb_im = im.convert('RGB')
    
    if gray:
        g_im = rgb_im.convert('L')
        input_NN = asarray(g_im)
                
    else:
    
        #Empty zeros matrix to be filled with pixel values of the image
        input_NN = np.zeros((3,224,224))

        #fill up the matrix with RGB values at each point
        for x in range(224):
            for y in range(224):
                r, g, b = rgb_im.getpixel((x,y))
                input_NN[0,x,y] = r
                input_NN[1,x,y] = g
                input_NN[2,x,y] = b

    #returns the 3D matrix a s a numpy array
    return input_NN


def show_image(Id,inputs):
    """Print image from Id of image in the input array"""
    im = plt.imshow(inputs[Id].astype('uint8'))



def get_class_dict():
    """Return dictionary of classes of each target"""

    class_dict_df = pd.read_csv("/kaggle/input/inputbirds/class_dict.csv")
    class_dict = {}
    for row in range(len(class_dict_df)):
        class_dict[class_dict_df.iloc[row][1]] = class_dict_df.iloc[row][0]
    return class_dict


def CNNDefinition(fil, k_size,  Poolsize, dense, 
                  input_shape, N_categ ,  loss = 'categorical_crossentropy',  
                  metrics = ['accuracy'], last_act = "softmax", lmb = 0.01, eta = 0.001,summary = True):
    """
    This CNNDefinition function allows the definition of a CNN network. The Network defined can be exploited for both 
    a regression and for a classification task.
    
    Parameters ---
    fil : list of the number of filters per each convolutional layer,
    k_size : dimension of the kernels related to each convolutional layer,
    Poolsize : dimension of the pooling layer applied after each convolutional one,
    dense : list of dimensions of dense layers applied after the convolutional step, 
    input_shape :  dimension of the input data , 
    N_categ : number of categories in the classification task (has to be set at 1 for regression),
    loss = 'categorical_crossentropy',  
    metrics = ['accuracy'], 
    last_act : default = "sigmoid", ok for classification. Has to be changed for regression
    """
    model = keras.Sequential()    
    model.add(layers.Conv2D(filters = fil[0], kernel_size = k_size[0],
                            padding = "same",
                            activation = "relu",
                            input_shape = input_shape,
                            kernel_regularizer=regularizers.l2(lmb)
                            ))
    model.add(layers.MaxPooling2D(Poolsize[0]))
#     model.add(layers.SpatialDropout2D(0.3))

    for i in range(1,len(fil)):
        model.add(layers.Conv2D(filters = fil[i], kernel_size = k_size[i],
                                padding = "same", activation = "relu",kernel_regularizer=regularizers.l2(lmb)))
        model.add(layers.MaxPooling2D(Poolsize[i]))
#         model.add(layers.SpatialDropout2D(0.3))

    model.add(layers.Flatten())
    for i in range(len(dense)):
        model.add(layers.Dense(dense[i],activation="relu",kernel_regularizer=regularizers.l2(lmb)))

    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(N_categ, activation= last_act))
    
    if summary:
        print(model.summary())

    #default optimizer is Adam, different learning rates affect the time the network takes to converge.
    
    #try different alogrithms and learning rates
    optimizer = keras.optimizers.Adam()
#     optimizer = keras.optimizers.SGD()
    
    optimizer.learning_rate.assign(eta)

    model.compile(
        optimizer = optimizer, 
        loss = loss, 
        metrics = metrics
    )

    return model



def Fit(model, X_train, Y_train, X_test, Y_test, pat = 100, mindelta = 0.01, batch = 64, epochs = 1200, verb = 1):
    
    '''
    Fit function for fitting the model. Training and test data are specified as parameters; 
    EarlyStopping is introduced to avoid overfitting.
    The function returns a dataframe containing the history of the training. The values of the accuracy and the different 
    epochs can be then inferred from such a returned quantity.
    
    Parameters ---
    model : model to be fit,
    X_train : train features,
    Y_train : train targets,
    X_test : test features,
    Y_test : test features,
    pat : default = 100, patience for the EarlyStopping, 
    mindelta : default = 0.01, mindelta in EarlyStopping, 
    batch : default = 64, batch size while fitting, 
    epochs : default = 1200, number of epochs for training,  
    verb = 1):
    '''

    early_stopping = keras.callbacks.EarlyStopping(
        patience = pat,
        min_delta = mindelta,
        restore_best_weights=True,
    )

    history = model.fit(
        X_train, Y_train,
        validation_data=(X_test, Y_test),
        batch_size = batch,
        epochs = epochs,
        callbacks=[early_stopping],
        verbose=verb, # hide the output because we have so many epochs
    )
    return pd.DataFrame(history.history)



In [ ]:
datagen_vgg = ImageDataGenerator() 
#
train_generator_vgg = datagen_vgg.flow_from_directory('/kaggle/input/100-bird-species/train',
                                                      batch_size=128,
                                                      target_size=(224,224),
                                                      class_mode='categorical')
#
validation_generator_vgg = datagen_vgg.flow_from_directory('/kaggle/input/100-bird-species/valid',
                                                           batch_size=128,
                                                           target_size=(224,224),
                                                           class_mode='categorical')

In [ ]:
# birds = pd.read_csv("/kaggle/input/inputbirds/birds.csv")
# label_list = list(birds.labels.unique())
# sub_label_list = label_list[0:100]
# birds_100 = birds[birds["labels"].isin(sub_label_list)]

# train_100 = birds_100[birds_100["data set"] == "train"]

# class_dict = get_class_dict()
# # labels = np.zeros((2987))
# labels = np.zeros((len(train_100)))
# for label in range(len(train_100)):
#     labels[label] = class_dict[train_100.iloc[label][1]]
    
# a_file = open("/kaggle/input/inputbirds/inputs_56_56_100.pkl", "rb")
# inputs = pickle.load(a_file)
# inputs = inputs/255

# train_size = 0.8
# test_size = 1 - train_size
# X_train, X_test, Y_train, Y_test = train_test_split(inputs, labels,  train_size=train_size,
#                                                     test_size=test_size)

# Y_train = to_categorical(Y_train)
# Y_test = to_categorical(Y_test)
# input_shape = inputs[0].shape

In [4]:
#code to read the train_20 birds subset with colors

birds = pd.read_csv("/kaggle/input/inputbirds/birds.csv")
label_list = list(birds.labels.unique())
sub_label_list = label_list[0:20]
birds_20 = birds[birds["labels"].isin(sub_label_list)]

train_20 = birds_20[birds_20["data set"] == "train"]

class_dict = get_class_dict()
labels = np.zeros((len(train_20)))
for label in range(len(train_20)):
    labels[label] = class_dict[train_20.iloc[label][1]]
    
a_file = open("/kaggle/input/inputbirds/inputs_56_56.pkl", "rb")
inputs = pickle.load(a_file)
inputs = inputs/255

train_size = 0.8
test_size = 1 - train_size
X_train, X_test, Y_train, Y_test = train_test_split(inputs, labels,  train_size=train_size,
                                                    test_size=test_size)

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
input_shape = inputs[0].shape

In [ ]:
#code to read the train_20 birds subset with gray scale

birds = pd.read_csv("/kaggle/input/inputbirds/birds.csv")
label_list = list(birds.labels.unique())
sub_label_list = label_list[0:20]
birds_20 = birds[birds["labels"].isin(sub_label_list)]

train_20 = birds_20[birds_20["data set"] == "train"]

class_dict = get_class_dict()
labels = np.zeros((len(train_20)))
for label in range(len(train_20)):
    labels[label] = class_dict[train_20.iloc[label][1]]
    
a_file = open("/kaggle/input/inputbirds/inputs_56_56_gray.pkl", "rb")
inputs = pickle.load(a_file)
inputs = inputs/255

inputs_corrected = np.zeros([len(train_20),56,56,1])
for n,image in enumerate(inputs):
    inputs_corrected[n] = np.array([image]).T


train_size = 0.8
test_size = 1 - train_size
X_train_g, X_test_g, Y_train_g, Y_test_g = train_test_split(inputs_corrected, labels,  train_size=train_size,
                                                    test_size=test_size)

Y_train_g = to_categorical(Y_train_g)
Y_test_g = to_categorical(Y_test_g)
input_shape_g = inputs_corrected[0].shape

In [ ]:
l_fil = [50,100] #number of filters randomly initialized -->
l_ker = [2,4] #filters -->  size of the square filter
l_pool = [5, 10]  #size of pooled matrix
l_dense = []
model_0 = CNNDefinition(l_fil, l_ker, l_pool, l_dense, X_train[0].shape, Y_train.shape[1], lmb = 0.00001, eta = 0.001)
history_0 = Fit(model_0, X_train, Y_train, X_test, Y_test, epochs = 150)

model_1 = CNNDefinition(l_fil, l_ker, l_pool, l_dense, X_train_g[0].shape, Y_train_g.shape[1], lmb = 0.00001, eta = 0.001)
history_1 = Fit(model_1, X_train_g, Y_train_g, X_test_g, Y_test_g, epochs = 150)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(history_0.accuracy,"--", color = "blue", label="Train Accuracy color")
plt.plot(history_1.accuracy,"--", color ="gray",label="Train Accuracy gray")
plt.plot(history_0.val_accuracy, color = "blue",label="Test accuracy color" )
plt.plot(history_1.val_accuracy, color ="gray", label="Test accuracy gray" )
plt.legend(fontsize=20)
plt.xlabel("Number of epochs",fontsize=20)
plt.ylabel("Accuracy",fontsize=20)
plt.savefig("graystudie.jpg")
plt.show()

In [ ]:
l_fil = [50,100] #number of filters randomly initialized -->
l_ker = [2,4] #filters -->  size of the square filter
l_pool = [5, 10]  #size of pooled matrix
l_dense = []
model_0 = CNNDefinition(l_fil, l_ker, l_pool, l_dense, X_train[0].shape, Y_train.shape[1], lmb = 0.00001, eta = 0.001)
history_0 = Fit(model_0, X_train, Y_train, X_test, Y_test, epochs = 150)

In [18]:
# CNN definition incuding droping

def CNNDefinition_(fil, k_size,  Poolsize, dense, 
                  input_shape, N_categ ,  loss = 'categorical_crossentropy',  
                  metrics = ['accuracy'], last_act = "softmax", lmb = 0.01, eta = 0.001,summary = True,drop1 = 0.2):
    """
    This CNNDefinition function allows the definition of a CNN network. The Network defined can be exploited for both 
    a regression and for a classification task.
    
    Parameters ---
    fil : list of the number of filters per each convolutional layer,
    k_size : dimension of the kernels related to each convolutional layer,
    Poolsize : dimension of the pooling layer applied after each convolutional one,
    dense : list of dimensions of dense layers applied after the convolutional step, 
    input_shape :  dimension of the input data , 
    N_categ : number of categories in the classification task (has to be set at 1 for regression),
    loss = 'categorical_crossentropy',  
    metrics = ['accuracy'], 
    last_act : default = "sigmoid", ok for classification. Has to be changed for regression
    """
    model = keras.Sequential()    
    model.add(layers.Conv2D(filters = fil[0], kernel_size = k_size[0],
                            padding = "same",
                            activation = "relu",
                            input_shape = input_shape,
                            kernel_regularizer=regularizers.l2(lmb)
                            ))
    model.add(layers.MaxPooling2D(Poolsize[0]))
#     model.add(layers.SpatialDropout2D(drop1))

    for i in range(1,len(fil)):
        model.add(layers.Conv2D(filters = fil[i], kernel_size = k_size[i],
                                padding = "same", activation = "relu",kernel_regularizer=regularizers.l2(lmb)))
        model.add(layers.MaxPooling2D(Poolsize[i]))
#         model.add(layers.SpatialDropout2D(drop))

    model.add(layers.Flatten())
    model.add(layers.Dropout(drop1))
    for i in range(len(dense)):
        model.add(layers.Dense(dense[i],activation="relu",kernel_regularizer=regularizers.l2(lmb)))


    model.add(layers.Dense(N_categ, activation= last_act))
    
    if summary:
        print(model.summary())

    #default optimizer is Adam, different learning rates affect the time the network takes to converge.
    
    #try different alogrithms and learning rates
    optimizer = keras.optimizers.Adam()
#     optimizer = keras.optimizers.SGD()
    
    optimizer.learning_rate.assign(eta)

    model.compile(
        optimizer = optimizer, 
        loss = loss, 
        metrics = metrics
    )

    return model



In [22]:
print("hello")

In [25]:
drop = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
l_fil = [50,100] #number of filters randomly initialized -->
l_ker = [2,4] #filters -->  size of the square filter
l_pool = [5, 10]  #size of pooled matrix
l_dense = []
lmb = 0.00001
eta = 0.001

train_accuracy = np.zeros(10)
test_accuracy = np.zeros(10)
n =  0
for d in tqdm(drop):
    model = CNNDefinition_(l_fil, l_ker, l_pool, l_dense, X_train[0].shape, Y_train.shape[1], lmb = lmb, eta = eta, drop1 = d)
    history = Fit(model, X_train, Y_train, X_test, Y_test, epochs = 500, verb = False)
    test_accuracy[n] = max(history.val_accuracy)
    train_accuracy[n] = max(history.accuracy)
    print(max(history.accuracy), max(history.val_accuracy), d)
    n+=1

In [29]:
plt.figure(figsize=(10,10))
plt.plot(drop,test_accuracy,"--", label="Test")
plt.plot(drop,train_accuracy,"--", label="Train")
plt.legend(fontsize=20)
plt.xlabel("Drop out",fontsize=20)
plt.ylabel("Accuracy",fontsize=20)
plt.savefig("drop_out_flattenlayer.jpg")
plt.show()



In [ ]:
history_1

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(history_0.accuracy,"--", label="DL Train Accuracy")
plt.plot(history_1.accuracy,"--", label="NDL Train Accuracy")
plt.plot(history_0.val_accuracy, label="DL Test accuracy" )
plt.plot(history_1.val_accuracy, label="NDL Test accuracy" )
plt.legend(fontsize=20)
plt.xlabel("Number of epochs",fontsize=20)
plt.ylabel("Accuracy",fontsize=20)
plt.savefig("densevsNodense.jpg")
plt.show()

In [7]:
#Main cell code to produce grid searches, I just mdoify the grid parameters at each time, so there is not a code for eac plot obtained, but you can guess how to do it with the commented lines


sns.set()

l_fil = [50,100] #number of filters randomly initialized -->
l_ker = [2,4] #filters -->  size of the square filter
l_pool = [5, 10]  #size of pooled matrix
l_dense = []
lmb = 0.00001
eta = 0.001

drop1 = np.array([0.1,0.2,0.3,0.4,0.5])
drop2 = np.array([0.1,0.2,0.3,0.4,0.5])

#train_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))
test_accuracy = np.zeros((len(drop1), len(drop2)))
# test_accuracy = np.zeros((len(pool_size_1), len(pool_size_2)))

for i in tqdm(range(len(drop1))):
    for j in tqdm(range(len(drop2))):
        
#         l_ker = [[filters_size_1[i], filters_size_1[i]],[filters_size_2[j], filters_size_2[j]]]
#         l_pool = [[pool_size_1[i],pool_size_1[i]],[pool_size_2[j],pool_size_2[j]]]
#         print(l_pool)
        
#         pool1 = pool_size_1[i]
#         pool2 = pool_size_2[j]
#         if int((int(56/pool1)/pool2)) == 0:
#             test_accuracy[i][j] = 0
            
#         else:
        print(drop1[i], drop2[j])
        model = CNNDefinition_(l_fil, l_ker,l_pool, l_dense, X_train[0].shape, Y_train.shape[1],lmb = lmb, eta = eta, summary = False, drop1 = drop1[i], drop2 =drop2[j])
        history = Fit(model, X_train, Y_train, X_test, Y_test, epochs = 100, verb = 0)
        print(max(history.val_accuracy))
        test_accuracy[i][j] = max(history.val_accuracy)

        
# fig, ax = plt.subplots(figsize = (10, 10))
# sns.heatmap(train_accuracy, annot=True, ax=ax, cmap="viridis")
# ax.set_title("Training Accuracy")
# ax.set_ylabel("$\eta$")
# ax.set_xlabel("$\lambda$")
# plt.show()

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(test_accuracy, annot=True, ax=ax, cmap="viridis",xticklabels=drop2, yticklabels=drop1)
ax.set_title("Test Accuracy")
# ax.set_xlabel("2nd pool size")
# ax.set_ylabel("1st pool size")
ax.set_ylabel("drop after 1st pooling")
ax.set_xlabel("drop after flatten layer")
plt.show()

In [13]:
fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(test_accuracy, annot=True, ax=ax, cmap="viridis",xticklabels=drop2, yticklabels=drop1)
ax.set_title("Test Accuracy",fontsize= 20)
# ax.set_xlabel("2nd pool size")
# ax.set_ylabel("1st pool size")
ax.set_ylabel("Spatial Drop out after 1st pooling", fontsize = 20)
ax.set_xlabel("Drop out after flatten layer",  fontsize = 20)
plt.savefig("dropout_grid.jpg")
plt.show()

In [ ]:
#code for the gray comparison

l_fil = [50,100] #number of filters randomly initialized -->
l_ker = [2,4] #filters -->  size of the square filter
l_pool = [5, 10]  #size of pooled matrix
l_dense = []
model = CNNDefinition(l_fil, l_ker, l_pool, l_dense, X_train[0].shape, Y_train.shape[1],lmb = 0.00001, eta = 0.001)
history_gray = Fit(model, X_train, Y_train, X_test, Y_test, epochs = 150)
plt.plot(history_gray.accuracy,"--")
plt.plot(history_gray.val_accuracy,"-")